# Custom CNN v/s EfficientNet Classifier Evaluation



In [26]:
# --- TEST & COMPARE: Custom CNN vs EfficientNet Functional Model on Validation Set ---

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, precision_score
import numpy as np
import os
import time

# Paths to models
CNN_H5_PATH = os.path.join('C:', os.sep, 'Users', 'ujwal', 'OneDrive', 'Documents', 'GitHub',
    '7thgear_Internship_Project', 'src', 'image_processing', 'classifier1_models', 'custom_cnn_classifier_model.h5')
FUNC_H5_PATH = os.path.join('C:', os.sep, 'Users', 'ujwal', 'OneDrive', 'Documents', 'GitHub',
    '7thgear_Internship_Project', 'src', 'image_processing', 'classifier1_models', 'efficientnet_functional_model.h5')

# Data and config
DATA_DIR = os.path.join('C:', os.sep, 'Users', 'ujwal', 'OneDrive', 'Documents', 'GitHub',
    '7thgear_Internship_Project', 'data', 'classifier1', 'augmented')
IMG_SIZE = (224, 224)
BATCH_SIZE = 8

# --- Custom CNN: Use rescale preprocessing ---
datagen_cnn = ImageDataGenerator(rescale=1./255, validation_split=0.2)
val_gen_cnn = datagen_cnn.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

start_cnn = time.time()
model_cnn = tf.keras.models.load_model(CNN_H5_PATH)
cnn_results = model_cnn.evaluate(val_gen_cnn, verbose=1, return_dict=True)
cnn_pred_probs = model_cnn.predict(val_gen_cnn)
cnn_pred = np.argmax(cnn_pred_probs, axis=1)
cnn_true = val_gen_cnn.classes
cnn_acc = accuracy_score(cnn_true, cnn_pred)
cnn_prec = precision_score(cnn_true, cnn_pred, average='weighted')
end_cnn = time.time()
cnn_time = end_cnn - start_cnn
cnn_size = os.path.getsize(CNN_H5_PATH) / 1024 / 1024  # MB

print(f"[Custom CNN] Loss: {cnn_results['loss']:.4f}, Accuracy: {cnn_acc:.4f}, Precision: {cnn_prec:.4f}, Time: {cnn_time:.2f}s, Size: {cnn_size:.2f}MB")

# --- EfficientNet Functional: Use EfficientNet preprocessing ---
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input

datagen_func = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2)
val_gen_func = datagen_func.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

start_func = time.time()
model_func = tf.keras.models.load_model(FUNC_H5_PATH)
model_func.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
func_results = model_func.evaluate(val_gen_func, verbose=1, return_dict=True)
func_pred_probs = model_func.predict(val_gen_func)
func_pred = np.argmax(func_pred_probs, axis=1)
func_true = val_gen_func.classes
func_acc = accuracy_score(func_true, func_pred)
func_prec = precision_score(func_true, func_pred, average='weighted')
end_func = time.time()
func_time = end_func - start_func
func_size = os.path.getsize(FUNC_H5_PATH) / 1024 / 1024  # MB

print(f"[EfficientNet Functional] Loss: {func_results['loss']:.4f}, Accuracy: {func_acc:.4f}, Precision: {func_prec:.4f}, Time: {func_time:.2f}s, Size: {func_size:.2f}MB")

# --- Comparison Table ---
print("\n--- Model Comparison ---")
print(f"{'Model':<25} {'Loss':>8} {'Accuracy':>10} {'Precision':>10} {'Time(s)':>10} {'Size(MB)':>10}")
print(f"{'Custom CNN':<25} {cnn_results['loss']:.4f} {cnn_acc:10.4f} {cnn_prec:10.4f} {cnn_time:10.2f} {cnn_size:10.2f}")
print(f"{'EfficientNet Functional':<25} {func_results['loss']:.4f} {func_acc:10.4f} {func_prec:10.4f} {func_time:10.2f} {func_size:10.2f}")

# Percentage differences
acc_diff = 100 * (func_acc - cnn_acc) / cnn_acc if cnn_acc else 0
prec_diff = 100 * (func_prec - cnn_prec) / cnn_prec if cnn_prec else 0
time_diff = 100 * (func_time - cnn_time) / cnn_time if cnn_time else 0
size_diff = 100 * (func_size - cnn_size) / cnn_size if cnn_size else 0
print(f"\nAccuracy diff: {acc_diff:+.2f}%\nPrecision diff: {prec_diff:+.2f}%\nTime diff: {time_diff:+.2f}%\nSize diff: {size_diff:+.2f}%")

Found 1027 images belonging to 2 classes.
129/129 [==============================] - 2s 13ms/step
[Custom CNN] Loss: 0.2487, Accuracy: 0.9825, Precision: 0.9830, Time: 4.38s, Size: 78.06MB
Found 1027 images belonging to 2 classes.


129/129 [==============================] - 5s 28ms/step
[EfficientNet Functional] Loss: 0.0939, Accuracy: 0.9864, Precision: 0.9867, Time: 13.90s, Size: 23.01MB

--- Model Comparison ---
Model                         Loss   Accuracy  Precision    Time(s)   Size(MB)
Custom CNN                0.2487     0.9825     0.9830       4.38      78.06
EfficientNet Functional   0.0939     0.9864     0.9867      13.90      23.01

Accuracy diff: +0.40%
Precision diff: +0.37%
Time diff: +217.04%
Size diff: -70.52%
